In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE212331"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE212331"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE212331.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE212331.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE212331.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Severity of lung function impairment drives transcriptional phenotypes of COPD and relates to immune and metabolic processes"
!Series_summary	"Gene expression profiles were generated from induced sputum samples in COPD and healthy controls. The study identified transcriptional phenotypes of COPD."
!Series_overall_design	"This study used unsupervised hierarchical clustering of induced sputum gene expression profiles of 72 stable COPD patients and 15 healthy controls to identify distinct and clinically relevant transcriptional COPD phenotypes."
Sample Characteristics Dictionary:
{0: ['tissue: sputum'], 1: ['disease group: COPD', 'disease group: Healthy Control'], 2: ['gold stage: 2', 'gold stage: 3', 'gold stage: 4', 'gold stage: 1', 'gold stage: n/a'], 3: ['age: 75', 'age: 66', 'age: 83', 'age: 70', 'age: 61', 'age: 77', 'age: 64', 'age: 81', 'age: 60', 'age: 62', 'age: 80', 'age: 65', 'age: 74', 'age: 73', 'age: 54', 'age: 67', 'age: 72', 'age: 71

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data from induced sputum samples
# The series title and summary explicitly mention "gene expression profiles" and not miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (COPD): Available in row 1 "disease group"
trait_row = 1

# Age: Available in row 3
age_row = 3

# Gender: Available in row 4
gender_row = 4

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert COPD trait values to binary (0: Healthy Control, 1: COPD)
    """
    if not isinstance(value, str):
        return None
    
    value = value.strip().lower()
    if "disease group:" in value:
        value = value.split("disease group:")[1].strip().lower()
    
    if "copd" in value:
        return 1
    elif "healthy control" in value or "control" in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age values to continuous numeric values
    """
    if not isinstance(value, str):
        return None
    
    value = value.strip().lower()
    if "age:" in value:
        value = value.split("age:")[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Convert gender values to binary (0: Female, 1: Male)
    """
    if not isinstance(value, str):
        return None
    
    value = value.strip().lower()
    if "gender:" in value:
        value = value.split("gender:")[1].strip().lower()
    
    if "female" in value or "f" in value:
        return 0
    elif "male" in value or "m" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait availability based on trait_row
is_trait_available = trait_row is not None

# Initial filtering and saving cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We proceed with this step since trait_row is not None
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The sample characteristics dictionary was shown in the previous output
    sample_chars = {
        0: ['tissue: sputum'], 
        1: ['disease group: COPD', 'disease group: Healthy Control'], 
        2: ['gold stage: 2', 'gold stage: 3', 'gold stage: 4', 'gold stage: 1', 'gold stage: n/a'], 
        3: ['age: 75', 'age: 66', 'age: 83', 'age: 70', 'age: 61', 'age: 77', 'age: 64', 'age: 81', 'age: 60', 
            'age: 62', 'age: 80', 'age: 65', 'age: 74', 'age: 73', 'age: 54', 'age: 67', 'age: 72', 'age: 71', 
            'age: 82', 'age: 69', 'age: 63', 'age: 76', 'age: 68', 'age: 78', 'age: 84', 'age: 88', 'age: 79', 
            'age: 24', 'age: 21', 'age: 20'], 
        4: ['gender: Female', 'gender: Male']
    }
    
    # Convert this dictionary to a DataFrame format that geo_select_clinical_features can process
    # We need to create a DataFrame where each column is a sample and each row is a characteristic
    # First, create a DataFrame with appropriate row indices
    clinical_data = pd.DataFrame()
    
    # Populate the DataFrame with the sample characteristics
    for row_idx, values in sample_chars.items():
        row_data = pd.Series(values)
        clinical_data[row_idx] = row_data
    
    # Transpose so each row is a characteristic
    clinical_data = clinical_data.transpose()
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to the specified output file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of extracted clinical features:
{0: [1.0, 75.0, 0.0]}
Clinical data saved to: ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE212331.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE212331/GSE212331_series_matrix.txt.gz


Gene data shape: (47231, 87)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina probe IDs (ILMN_* format) and not human gene symbols
# They need to be mapped to gene symbols for proper gene expression analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene identifiers and gene symbols in the annotation data
# Based on the preview, we can see:
# - 'ID' column contains Illumina probe IDs (ILMN_*) which match our gene expression data indices
# - 'Symbol' column contains gene symbols 

# 2. Get gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene-level expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and standardize
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalizing gene symbols, expression data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene data to the specified output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe shape: (44837, 2)
Sample of gene mapping:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB


Gene-level expression data shape: (21372, 87)
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


After normalizing gene symbols, expression data shape: (20259, 87)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Gene expression data saved to: ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE212331.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (20259, 87)
Gene data column names (sample IDs):
Index(['GSM6524456', 'GSM6524458', 'GSM6524459', 'GSM6524460', 'GSM6524462'], dtype='object')



Raw clinical data structure:
Clinical data shape: (5, 88)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM6524456', 'GSM6524458', 'GSM6524459',
       'GSM6524460'],
      dtype='object')

Sample characteristics dictionary:
{0: ['tissue: sputum'], 1: ['disease group: COPD', 'disease group: Healthy Control'], 2: ['gold stage: 2', 'gold stage: 3', 'gold stage: 4', 'gold stage: 1', 'gold stage: n/a'], 3: ['age: 75', 'age: 66', 'age: 83', 'age: 70', 'age: 61', 'age: 77', 'age: 64', 'age: 81', 'age: 60', 'age: 62', 'age: 80', 'age: 65', 'age: 74', 'age: 73', 'age: 54', 'age: 67', 'age: 72', 'age: 71', 'age: 82', 'age: 69', 'age: 63', 'age: 76', 'age: 68', 'age: 78', 'age: 84', 'age: 88', 'age: 79', 'age: 24', 'age: 21', 'age: 20'], 4: ['gender: Female', 'gender: Male']}

Values in trait row:
['!Sample_characteristics_ch1' 'disease group: COPD' 'disease group: COPD'
 'disease group: COPD' 'disease group: COPD']

Created clinical features dataframe:
Shape: (1, 87)
                

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (87, 20260)
For the feature 'Chronic_obstructive_pulmonary_disease_(COPD)', the least common label is '0' with 15 occurrences. This represents 17.24% of the dataset.
The distribution of the feature 'Chronic_obstructive_pulmonary_disease_(COPD)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE212331.csv
